# Trabalho BI - Tópicos Avançados
## A base do trabalho contém todas as reclamações fundamentadas que foram audiência no ano de 2017.
### O notebook cria as seguintes WordClouds com os 40 serviços mais reclamados, utilizando a coluna 'serviço' :
#### - Criação da WordCloud utilizando Expressões Regulares para Pré-processamento.
#### - Criação da WordCloud utilizando Expressões Regulares, Stopwords, Remoção de Acentuação e Tokenização para Pré-processamento.

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Carregamento da base
data = pd.read_csv(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\base reclamacoes Procon.csv', sep = ';', encoding ='windows-1252', keep_default_na = True)

In [ ]:
# Existem valores nulos (por coluna)?
data.isna().sum()

In [ ]:
#Remoção de linhas com valores nulos
data = data.dropna(subset=['empresa','problema']).astype(str)


## WordCloud com os 40 serviços mais reclamados no Procon em 2017

### - Criação da WordCloud utilizando Expressões Regulares para Pré-processamento.

In [ ]:
# Criação de dataframe com a coluna serviço
data_servico_1 = pd.DataFrame()
data_servico_1 = pd.DataFrame({'servico':data.serviço.str.lower()}) # ---> Formatação de todas as palavras em minúsculas

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario = {'/':' ',',':'','\(':'','\)':'',"-":'','\.':'',r"^\t":'','–':'','\ etc.':'','\.com':'',' ':'_'} 
data_servico_1 = data_servico_1.replace({'servico': dicionario},regex=True)

In [ ]:
# Criação de uma string única com todas as ocorrências dos nomes das empresas
string_servico = " ".join([text for text in data_servico_1['servico']])

### - Criação da WordCloud utilizando Expressões Regulares, Stopwords, Tokenização e Ngrams para Pré-processamento

In [ ]:
# Criação de dataframe com a coluna empresa
data_servico_2 = pd.DataFrame({'servico':data.serviço.str.lower()}) # ---> Formatação de todas as palavras em minúsculas

In [ ]:
# Utilização de expressões regulares para limpeza do texto
dicionario = {'/':' ',',':'','\(':'','\)':'',"-":'','\.':'',r"^\t":'','–':''}     # ---> Remoção de ' ':'_'
data_servico_2 = data_servico_2.replace({'servico': dicionario},regex=True)

In [ ]:
# Expressões regulares - remoção de todas as palavras com números e pontuação
alphanumeric = lambda x: re.sub(r"""\w*\d\w*""", ' ', x)
punc = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)

data_servico_2['servico'] = data_servico_2.servico.map(alphanumeric).map(punc)

In [ ]:
# Remoção de acentos
data_servico_2['servico'] = data_servico_2.servico.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
# Tokenização da coluna serviço
data_servico_2['servico'] = data_servico_2.servico.map(word_tokenize)

In [ ]:
# Remoção de StopWords
stopword_list = nltk.corpus.stopwords.words('portuguese')
new_stopwords = ['d','co','kd','texto','object','dtype','i','c','outros','etc']
stopword_list.extend(new_stopwords)

stop_lambda = lambda x: [y for y in x if y not in stopword_list]

data_servico_2['servico'] = data_servico_2.servico.map(stop_lambda)

In [ ]:
# Criação de lista com as listas de tokens
tokens_list = [] 
for row in data_servico_2.servico:
    tokens_list.append(row)


In [ ]:
# Concatenação dos tokens em ngrams
ngram_list = []
for words in tokens_list:
    ngram_list.append(' '.join(words))

In [ ]:
# Criação de dicinário com a frequência dos ngrams
words = ngram_list
words_freq = {} 
for items in words: 
    words_freq[items] = words.count(items)

## WordClouds

In [ ]:
# Definição de imagem para formato da Wordcloud todas as empresas 
brazil_mask = np.array(Image.open(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\brazil-map.png'))
img=mpimg.imread(r'C:\Users\rfsantos\OneDrive - Mongeral Aegon\2020\Modulos\BI\NLP\Trabalho\Arquivos fonte finais\brazil-map.png')
imgplot = plt.imshow(img)
plt.axis("off");

In [ ]:
# Criação das WordClouds 
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(20,10), sharex=True, sharey=True)

## Utilização Expressões Regulares para Pré-processamento
cloud1 = WordCloud(background_color='black',width=1600, height=800,max_font_size=200,max_words=40, mask=brazil_mask,collocations=False,contour_width=1, contour_color='steelblue').generate(string_servico)
fig.add_subplot(ax1)
plt.gca().imshow(cloud1)
plt.gca().set_title('Regex', fontsize=20)
plt.gca().axis('off')

## Utilização de Expressões Regulares, Stopwords, Remoção de Acentuação e Tokenização para Pré-processamento
cloud2 = WordCloud(background_color='black',width=1600,height=800,max_font_size=200,max_words=40, mask=brazil_mask,contour_width=1, contour_color='steelblue').generate_from_frequencies(words_freq)
fig.add_subplot(ax2)
plt.gca().imshow(cloud2)
plt.gca().set_title('Regex, Stopwords, Remoção de Acentuação e Tokenização', fontsize=20)
plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.savefig('serviços-total.png')
plt.show()